In [1]:
import pandas as pd
import numpy as np

In [2]:
auctions = pd.read_csv('data/train_auctions.csv.gzip')

In [3]:
auctions.head().T

,0,1,2,3,4
created,2019-03-13 01:43:33.445,2019-03-13 04:58:35.078,2019-03-13 04:20:57.666,2019-03-13 04:20:57.698,2019-03-10 22:24:56.684
application_id,0,0,0,0,0
ref_type,1891515180541284343,1891515180541284343,1891515180541284343,1891515180541284343,1891515180541284343
ref_hash,8464844987297247076,3250564871270161533,7953318831018100268,7953318831018100268,8355110941557237501
click_hash,NaN,NaN,NaN,NaN,NaN
attributed,False,False,False,False,False
implicit,False,False,True,False,True
device_countrycode,6333597102633388268,6333597102633388268,6333597102633388268,6333597102633388268,6333597102633388268
device_brand,3.08306e+17,3.08306e+17,5.13799e+17,5.13799e+17,1.08337e+18
device_model,4.44525e+18,4.44525e+18,6.02654e+18,6.02654e+18,7.87661e+18


In [5]:
auctions.dtypes

created                object
application_id          int64
ref_type                int64
ref_hash                int64
click_hash            float64
attributed               bool
implicit                 bool
device_countrycode      int64
device_brand          float64
device_model          float64
session_user_agent     object
user_agent             object
event_uuid             object
kind                   object
wifi                   object
trans_id               object
ip_address              int64
device_language       float64
dtype: object

In [8]:
len(auctions)

3412

In [1]:
#auctions.describe()

In [13]:
#auctions.memory_usage(deep=True)

In [10]:
auctions.size

61416

In [12]:
auctions.isnull().sum()

created                  0
application_id           0
ref_type                 0
ref_hash                 0
click_hash            3412
attributed               0
implicit                 0
device_countrycode       0
device_brand          2365
device_model             1
session_user_agent      48
user_agent            1683
event_uuid            2547
kind                  2547
wifi                  1683
trans_id              3406
ip_address               0
device_language         34
dtype: int64

In [14]:
auctions['created'].count()

3412

In [19]:
# device_languaje tiene algunos grandes grupos
auctions['device_language'].value_counts()

3.301378e+18    1484
9.028383e+18     714
4.621024e+18     632
6.977049e+18     238
4.077062e+17      70
1.526421e+18      56
4.060930e+18      48
2.822843e+17      39
6.035180e+18      24
1.193279e+18      21
6.535228e+18      10
1.101539e+17       9
3.177265e+18       5
3.695787e+17       5
3.095856e+18       4
6.569908e+18       2
5.846217e+18       2
7.774316e+18       2
4.594650e+18       2
6.782349e+18       1
5.061642e+18       1
1.686906e+18       1
5.794427e+17       1
2.488366e+18       1
3.544544e+18       1
3.074904e+18       1
2.061937e+18       1
4.073835e+18       1
6.846873e+18       1
7.045130e+18       1
Name: device_language, dtype: int64

In [18]:
# device_brand tiene algunos grandes grupos
auctions['device_brand'].value_counts()

3.083059e+17    472
2.208835e+18    185
2.523246e+18    116
5.951325e+18     72
2.987569e+18     40
3.812621e+18     37
5.137992e+17     34
6.538562e+18     26
3.228516e+18      9
1.083369e+18      9
4.371308e+18      7
3.093166e+18      5
2.262848e+18      5
4.994101e+18      4
3.849490e+18      4
8.285870e+18      3
3.742387e+18      3
5.645692e+18      3
8.230045e+17      3
6.111647e+18      3
9.169527e+18      1
5.588755e+18      1
3.172231e+18      1
5.189983e+18      1
3.624089e+18      1
7.855186e+18      1
2.748831e+18      1
Name: device_brand, dtype: int64

In [17]:
auctions['ref_type'].value_counts()

1891515180541284343    2080
1494519392962156891    1332
Name: ref_type, dtype: int64

In [42]:
auctions.loc[auctions['device_model'].isnull()]

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
2157,2019-03-09 16:08:19.919,30,1494519392962156891,4364935818432173881,NaN,False,True,6333597102633388268,NaN,NaN,HasOffers Mobile AppTracking v1.0,NaN,78082c4c-0fe1-4fc7-a718-4eba1d943b9c,Open,NaN,NaN,2208800741316080623,NaN


In [55]:
#auctions['device_model'].value_counts()

In [17]:
# Tabla de valores unicos en device_languaje
#auctions.loc[auctions['device_language'].isnull()]

In [51]:
#auctions.loc[auctions['session_user_agent'].isnull()]
auctions['session_user_agent'].describe()

count             3364
unique              12
top       http-kit/2.0
freq              1729
Name: session_user_agent, dtype: object

In [16]:
# Muchos valores repetidos y muy pocos unicos o con grupos chicos
auctions['session_user_agent'].value_counts()

http-kit/2.0                                                                                                                                                                                                                                                                  1729
adjust.com                                                                                                                                                                                                                                                                    1592
HasOffers Mobile AppTracking v1.0                                                                                                                                                                                                                                               22
Mozilla/5.0 (iPhone; CPU iPhone OS 12_1_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Mobile/15E148 Safari/604.1                                      

In [25]:
# grupos grandes, se puede pasar a un tipo diferente a int64 para que no ocupe tanta memoria
auctions['application_id'].value_counts()

7     947
9     731
10    389
16    362
8     328
2     174
29     97
20     95
6      35
1      34
26     28
34     28
15     20
0      18
3      17
28     17
12     15
18     14
24     13
17     13
14     13
21      7
5       3
13      3
33      2
30      2
19      2
32      2
36      1
4       1
23      1
Name: application_id, dtype: int64

In [47]:
# Cambio el tipo de datos a uno mas chico ya que los valores van del 0 al 36
auctions['application_id'] = auctions['application_id'].astype(np.int8)

In [27]:
# son solo 2 grupos!!
auctions['ref_type'].value_counts()

1891515180541284343    2080
1494519392962156891    1332
Name: ref_type, dtype: int64

In [29]:
auctions['attributed'].value_counts()
# Son todos falsos

False    3412
Name: attributed, dtype: int64

In [30]:
auctions['implicit'].value_counts()

False    2547
True      865
Name: implicit, dtype: int64

In [32]:
auctions['trans_id'].value_counts()

{hash}                                  3
44aeb6df-8db4-4b3f-ac67-0a802d479c86    1
89bda305-bff1-4c22-ae23-1a237eac0002    1
22380348598                             1
Name: trans_id, dtype: int64

In [38]:
# los 6 valores no nulos, que significa {hash}
auctions.loc[auctions['trans_id'].notnull()]

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
2234,2019-03-09 19:48:10.457,16,1891515180541284343,6173899971792625142,NaN,False,False,6333597102633388268,2.208835e+18,9.109466e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 7.1.1; Moto G ...,NaN,NaN,True,{hash},6586849487961128061,9.028383e+18
3249,2019-03-12 13:00:45.072,16,1891515180541284343,6808608047290129862,NaN,False,False,6333597102633388268,2.208835e+18,8.716277e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 7.0; Moto G (4...,NaN,NaN,True,{hash},6799929828891198374,9.028383e+18
3267,2019-03-12 22:27:58.098,16,1891515180541284343,5517347051664000025,NaN,False,False,6333597102633388268,3.083059e+17,2.066431e+17,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; SM-G570...,NaN,NaN,True,{hash},3162368506662488065,9.028383e+18
3380,2019-03-06 17:01:26.123,1,1494519392962156891,5634738659911451938,NaN,False,False,6333597102633388268,5.951325e+18,8.354574e+18,Mozilla/5.0 (iPhone; CPU iPhone OS 12_1 like M...,NaN,NaN,NaN,NaN,44aeb6df-8db4-4b3f-ac67-0a802d479c86,5145638343291847724,4.621024e+18
3409,2019-03-05 11:03:58.537,19,1494519392962156891,8280469812012804019,NaN,False,False,6333597102633388268,5.951325e+18,4.349816e+18,Mozilla/5.0 (iPhone; CPU iPhone OS 12_1_4 like...,NaN,NaN,NaN,NaN,89bda305-bff1-4c22-ae23-1a237eac0002,543629953276043235,4.621024e+18
3410,2019-03-07 13:42:47.814,19,1494519392962156891,1530864130516519187,NaN,False,False,6333597102633388268,5.951325e+18,1.764701e+18,Mozilla/5.0 (iPhone; CPU iPhone OS 12_1_2 like...,NaN,NaN,NaN,NaN,22380348598,5929364725345188154,1.526421e+18


In [43]:
auctions.isnull().sum()

created                  0
application_id           0
ref_type                 0
ref_hash                 0
click_hash            3412
attributed               0
implicit                 0
device_countrycode       0
device_brand          2365
device_model             1
session_user_agent      48
user_agent            1683
event_uuid            2547
kind                  2547
wifi                  1683
trans_id              3406
ip_address               0
device_language         34
dtype: int64

In [45]:
auctions['kind'].value_counts() # esto es lo interesante

af_app_opened                 428
app_open                      212
EVENT_Homepage                 70
open                           36
EVENT_OPEN_APP                 33
Open                           22
login_success                  17
account_summary_first_step     13
EVENT_HOMEPAGE                  6
EVENT_LISTINGVIEW_FLIGHT        6
deeplink                        5
reattribution                   4
registration                    3
payment method add              3
journey first drop off          2
af_list_view                    1
install                         1
EVENT_PROMO                     1
journey reserved                1
journey drop off                1
Name: kind, dtype: int64

In [91]:
# Los eventos los describo todos en mayusculas
auctions.loc[auctions['kind'] == 'event_homepage'] = 'EVENT_HOMEPAGE'

In [92]:
auctions.loc[auctions['kind'] == 'Open'] = 'open'

In [41]:
auctions['device_countrycode'].value_counts() # solo 2 paises

6333597102633388268    1820
2970470518450881158    1592
Name: device_countrycode, dtype: int64

In [82]:
#auctions['created'].str[:19] # si lo asigno a la columna me limpia los datos

In [84]:
pd.to_datetime(auctions['created'].str[:19], format='%Y-%m-%d %H:%M:%S').head()

0   2019-03-13 01:43:33
1   2019-03-13 04:58:35
2   2019-03-13 04:20:57
3   2019-03-13 04:20:57
4   2019-03-10 22:24:56
Name: created, dtype: datetime64[ns]

In [6]:
auctions.head(20)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,2019-03-13 01:43:33.445,0,1891515180541284343,8464844987297247076,NaN,False,False,6333597102633388268,3.083059e+17,4.445252e+18,NaN,NaN,NaN,NaN,NaN,NaN,5529284367937637301,NaN
1,2019-03-13 04:58:35.078,0,1891515180541284343,3250564871270161533,NaN,False,False,6333597102633388268,3.083059e+17,4.445252e+18,NaN,NaN,NaN,NaN,NaN,NaN,6098022721363533304,NaN
2,2019-03-13 04:20:57.666,0,1891515180541284343,7953318831018100268,NaN,False,True,6333597102633388268,5.137992e+17,6.026543e+18,HasOffers Mobile AppTracking v1.0,NaN,0bfa9b4c-e07a-4522-b32d-138e3165ac98,Open,NaN,NaN,4636407223847323867,3.301378e+18
3,2019-03-13 04:20:57.698,0,1891515180541284343,7953318831018100268,NaN,False,False,6333597102633388268,5.137992e+17,6.026543e+18,NaN,NaN,NaN,NaN,NaN,NaN,4636407223847323867,NaN
4,2019-03-10 22:24:56.684,0,1891515180541284343,8355110941557237501,NaN,False,True,6333597102633388268,1.083369e+18,7.876612e+18,HasOffers Mobile AppTracking v1.0,NaN,8f235a8c-41bf-4399-b33d-c41577c949b0,Open,NaN,NaN,685400811232710215,3.301378e+18
5,2019-03-10 22:24:57.545,0,1891515180541284343,8355110941557237501,NaN,False,False,6333597102633388268,1.083369e+18,7.876612e+18,NaN,NaN,NaN,NaN,NaN,NaN,685400811232710215,NaN
6,2019-03-11 03:47:22.802,0,1891515180541284343,4306303321567254439,NaN,False,True,6333597102633388268,3.083059e+17,9.179743e+18,HasOffers Mobile AppTracking v1.0,NaN,b3f7905b-5058-4eca-853c-b3c670d7ac54,Open,NaN,NaN,7462893331703692427,4.073835e+18
7,2019-03-11 21:21:05.986,0,1891515180541284343,1670430602051291592,NaN,False,False,6333597102633388268,1.083369e+18,7.551685e+17,NaN,NaN,NaN,NaN,NaN,NaN,5634136297886425939,NaN
8,2019-03-11 21:21:05.855,0,1891515180541284343,1670430602051291592,NaN,False,True,6333597102633388268,1.083369e+18,7.551685e+17,HasOffers Mobile AppTracking v1.0,NaN,28a6e6a0-ba70-46a2-a63f-1e74b0dd5235,Open,NaN,NaN,5634136297886425939,3.301378e+18
9,2019-03-13 08:29:31.875,2,1891515180541284343,5651666841804012468,NaN,False,False,6333597102633388268,3.083059e+17,8.656854e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 22; SM-J200M B...,NaN,NaN,True,NaN,5997848429649592955,9.028383e+18
